# Demo of `COPY FROM` to rapidly ingest data into Kùzu

This notebook demonstrates how to use the `COPY FROM` command to load data from the following sources:

- CSV files
- Subquery results
- Pandas DataFrames

## Create a Kùzu database

The first step is to import `kuzu`, create a database and start a connection to it.


In [1]:
import kuzu
import shutil

DB_NAME = "./db"
shutil.rmtree(DB_NAME, ignore_errors=True)
db = kuzu.Database(DB_NAME)
conn = kuzu.Connection(db)

## Create node tables

The node tables are created using the following schemas.

In [2]:
conn.execute(
    """
    CREATE NODE TABLE Person (
        id STRING,
        name STRING,
        age INT64,
        PRIMARY KEY (id)
    )
    """
)

conn.execute(
    """
    CREATE NODE TABLE City (
        id STRING,
        city STRING,
        PRIMARY KEY (id)
    )
    """
)

conn.execute(
    """
    CREATE NODE TABLE Company (
        id STRING,
        company STRING,
        PRIMARY KEY (id)
    )
    """
)

## Create relationship tables
The following cells creates relationship tables with the required `FROM` and `TO` relationships.

In [3]:
conn.execute(
    """
    CREATE REL TABLE LivesIn (
        FROM Person
        TO City
    )
    """
)

conn.execute(
    """
    CREATE REL TABLE HeadquarteredIn (
        FROM Company
        TO City
    )
    """
)

conn.execute(
    """
    CREATE REL TABLE WorksAt (
        FROM Person
        TO Company
    )
    """
)

## Use `COPY FROM` to ingest data from files

The `COPY FROM` technique can be conveniently used to ingest data from multiple CSV files as shown below.

In [4]:
# Copy node tables
conn.execute("COPY Person FROM 'data/person.csv' (header=true)")
conn.execute("COPY City FROM 'data/city.csv'")
conn.execute("COPY Company FROM 'data/company.csv'")

# Copy relationship tables
conn.execute("COPY LivesIn FROM 'data/lives_in.csv'")
conn.execute("COPY HeadquarteredIn FROM 'data/headquartered_in.csv'")
conn.execute("COPY WorksAt FROM 'data/works_at.csv'")

In [5]:
# Give me the names and ages of all people who live in San Francisco
result = conn.execute(
    """
    MATCH (p:Person)-[l:LivesIn]->(c:City)
    WHERE c.city = "San Francisco"
    RETURN p.name AS name, p.age AS age
    """
)
while result.has_next():
    print(result.get_next())

['Lisa', 25]
['James', 28]


## Use `COPY FROM` to ingest data via a subquery

Another way to ingest data is to pass the results of a subquery to the `COPY FROM` command. In the below
example, we first scan the data from the CSV file, to inspect its contents. This is done using the `LOAD FROM`
statement. Note that `LOAD FROM` does not insert the data into a table - it simply scans the contents of the
source. The `COPY FROM` command is then used to insert the data into the `nodes` table.


In [6]:
# Create a new table called `Person2`
conn.execute(
    """
    CREATE NODE TABLE Person2 (
        id STRING,
        name STRING,
        age INT64,
        PRIMARY KEY (id)
    )
    """
)

In [7]:
# Check out the contents of the CSV file using the `LOAD FROM` command
res = conn.execute(
    """
    LOAD FROM 'data/person.csv' (header = true)
    RETURN id, name, CAST(age, "INT64")
    """
)
while res.has_next():
    print(res.get_next())

['p1', 'Lisa', 25]
['p2', 'James', 28]
['p3', 'Carol', 32]
['p4', 'David', 31]


In [8]:
# Now, pass the contents of the CSV file to COPY the data to the new table `Person2` using a subquery
conn.execute(
    """
    COPY Person2 FROM (
        LOAD FROM 'data/person.csv' (header = true)
        RETURN id, name, CAST(age, "INT64"
        )
    )
    """
)

In [9]:
conn.execute("DROP TABLE Person2")

## Use `COPY FROM` to ingest data from a DataFrame

If you already have data in a DataFrame as you wrangle it in libraries like Pandas or Polars,
you can use the `COPY FROM` command on the DataFrame to ingest the data into Kùzu. At the time of
writing this, the `COPY FROM` command only supports copying from Pandas DataFrames via a subquery,
but from Kùzu 0.5.0 and beyond, two additional features will be supported:

- `COPY FROM` Polars DataFrames and Arrow Tables
- `COPY FROM` a DataFrame directly (either Pandas or Polars)

The cells below demonstrate how to use the `COPY FROM` command to ingest data from a Pandas DataFrame
via subquery. Stay tuned for the upcoming features in Kùzu 0.5.0 and beyond!

In [10]:
# Create a new table called `Person3`
conn.execute(
    """
    CREATE NODE TABLE Person3 (
        id STRING,
        name STRING,
        age INT64,
        PRIMARY KEY (id)
    )
    """
)

In [11]:
# Check out the contents of the CSV file using the `LOAD FROM` command
res = conn.execute(
    """
    LOAD FROM 'data/person.csv' (header = true)
    RETURN id, name, CAST(age, "INT64") AS age
    """
)
df = res.get_as_df()
df

,id,name,age
0,p1,Lisa,25
1,p2,James,28
2,p3,Carol,32
3,p4,David,31


In [12]:
# Now, pass the contents of the CSV file to COPY the data to the new table `Person3` using a subquery
conn.execute("COPY Person3 FROM (LOAD FROM df RETURN * )")

In [13]:
conn.execute(
    """
    MATCH (p:Person)
    WHERE p.age < 30
    RETURN p.id, p.name, p.age
    """
).get_as_df()

,p.id,p.name,p.age
0,p1,Lisa,25
1,p2,James,28


In [14]:
conn.execute("DROP TABLE Person3")

## Conclusions

This notebook covered the basics of ingesting data into Kùzu from files, subquery results and DataFrames.

Check out our documentation for more information: https://docs.kuzudb.com

Only very basic Cypher queries were shown. More advanced Cypher will be covered in future notebooks!